This Notebook evaluates a convergence study for the Bow Shock Problem as presented presented in the phd thesis "Implicit Discontinuous Galerkin Shock Tracking methods for Compressible Flows with Shocks" (Vandergrift 2024).

In [1]:
#r "BoSSSpad.dll"
#r "XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


init workflow


In [2]:
BoSSSshell.WorkflowMgm.Init("CNS_AcousticWave1D_ConvStudy");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


{ Session Count = 60; Grid Count = 60; Path = \\dc3\userspace\sebastian\cluster\CNS_AcousticWave1D_ConvStudy }

In [3]:
var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var database = OpenOrCreateDatabase(dbPath);
var dt = database.Sessions.Last().CreationTime;
dt


2024-05-03 10:54:50Z

## Choose which runs to plot

In [4]:
var ses=database.Sessions.Where(s => (s.Name.Contains("sP1.5") &&s.Name.Contains("ampneg0")&&s.Name.Contains("amppos1E-05")&& !s.Name.Contains("241"))); // the one plotted in the publication
ses.OrderBy(si => si.GetGrids().Pick(0).NumberOfCells);
ses


#0: AcousticWave	AW_p3_xCells121_yCells3_sP1.5_pST10_wP-0.8_ampneg0_amppos1E-05_wL0.8_Mach1.5_1sinus_RK4	5/3/2024 10:59:46 AM	9aa361cb...
#1: AcousticWave	AW_p3_xCells61_yCells3_sP1.5_pST10_wP-0.8_ampneg0_amppos1E-05_wL0.8_Mach1.5_1sinus_RK4	5/3/2024 10:59:32 AM	5d9caf6a...
#2: AcousticWave	AW_p3_xCells31_yCells3_sP1.5_pST10_wP-0.8_ampneg0_amppos1E-05_wL0.8_Mach1.5_1sinus_RK4	5/3/2024 10:59:14 AM	43c9286c...
#3: AcousticWave	AW_p2_xCells121_yCells3_sP1.5_pST10_wP-0.8_ampneg0_amppos1E-05_wL0.8_Mach1.5_1sinus_RK4	5/3/2024 10:58:27 AM	e6fe2b8a...
#4: AcousticWave	AW_p3_xCells16_yCells3_sP1.5_pST10_wP-0.8_ampneg0_amppos1E-05_wL0.8_Mach1.5_1sinus_RK4	5/3/2024 10:58:58 AM	75dcede5...
#5: AcousticWave	AW_p2_xCells61_yCells3_sP1.5_pST10_wP-0.8_ampneg0_amppos1E-05_wL0.8_Mach1.5_1sinus_RK4	5/3/2024 10:58:09 AM	efb9a79d...
#6: AcousticWave	AW_p2_xCells31_yCells3_sP1.5_pST10_wP-0.8_ampneg0_amppos1E-05_wL0.8_Mach1.5_1sinus_RK4	5/3/2024 10:57:54 AM	db6c132e...
#7: AcousticWave	AW_p2_xCells16_yCells3

# Helper Functions to obtain the error metrics

In [5]:
(double[] xVals, double[]yVals, double tPhys) GetPlotAlongTRay(double xMin,double xMax, double t, int nRef,string field,ISessionInfo si, double t0){
//double shockPosition=0.5;
var p1= new double[] {xMin,0.0015};
var p2=new double[] {xMax,0.0015};

//closest timestep to time t
var tiEnd=si.Timesteps.Where(ti =>ti.PhysicalTime >t-1e-2).OrderBy(ti => Math.Abs(ti.PhysicalTime - t))
            .FirstOrDefault();

var pEnd =tiEnd.GetField(field);
double[] yVals=pEnd.EvaluateAlongLine(p1,p2,nRef);

// substract the t0 Base to only obtain the pertuabtions
var ti=si.Timesteps.Where(ti =>Math.Abs(ti.PhysicalTime - t0)<1e-1).OrderBy(ti => Math.Abs(ti.PhysicalTime - t0))
            .FirstOrDefault();
var tOField=ti.GetField(field);
double[] yVals_tOField=tOField.EvaluateAlongLine(p1,p2,nRef);
yVals.AccV(-1.0,yVals_tOField);

double[] xVals= GenericBlas.Linspace(xMin,xMax,nRef);
return(xVals,yVals,tiEnd.PhysicalTime);
}

Number of cells

In [32]:
double[] Cells = ses.Where(si => si.Name.Contains("p0")).Select(si => (double) si.GetGrids().Pick(0).NumberOfCells).Distinct().Order().ToArray();
Cells

[ 48, 93, 183, 363 ]

cell lengths in $x$-direction

In [42]:
var Hs= ses.Select(si => si.Timesteps.Last().Fields.Pick(0).Basis.GridDat.iGeomCells.GetCellVolume(0)/0.01).Distinct().OrderDescending().ToArray();
Hs

[ 0.1875, 0.0967741935483871, 0.04918032786885246, 0.024793388429752067 ]

helper arrays

In [43]:
int[] degs = new int[] {0,1,2,3};
//degs = new int[] {0};
var errors_array= MultidimensionalArray.Create(degs.Count(),ses.Count());


Function used to calculate the EOCs

In [44]:
double ApproxSlope(List<double> errors, List<double> cellSize) {
    var slopes = new List<double>();
    double EOC;
    for (int i = 0; i < errors.Count - 1; i++) {
        double ratio = errors[i] / errors[i + 1];
        double slope = Math.Log(ratio) / Math.Log(cellSize[i] / cellSize[i + 1]);
        slopes.Add(slope);
    }
    if (slopes.Count > 0) {
        EOC = slopes.Average();
    } else {
        EOC = 0;
    }
    return EOC;
}

Function used to compute the ideal convergence plot

In [45]:
List<double> GetIdealConvPlot(List<double> errors,List<double> cellSize, int p){
        var idealErrors = new List<double>();

    // Calculate ideal errors for each cell size
    for (int i = 0; i < cellSize.Count; i++)
    {
        double idealError = errors[0] * Math.Pow(cellSize[i] / cellSize[0], p + 1);
        idealErrors.Add(idealError);
    }

    return idealErrors;
} 

compute the exact analytical pressure amplification/reduction factor for both cases investigated

In [46]:
// Given variables
var MBaseL = 1.5; // Base Mach number on the left
var gamma = 1.4; //Specific heat ratio

// Calculate right Mach number (Mach_R)
var MBaseR  = Math.Sqrt((1 + ((gamma-1)/2) * MBaseL*MBaseL) / (gamma * MBaseL*MBaseL - (gamma-1)/2));

// Upstream fast acoustic wave amplification coefficient
var delta_pPrR_over_delta_pPrL = ((1+MBaseL)*(1+MBaseL)) / (1+2*MBaseR+1/MBaseL/MBaseL) * (1 - ((gamma-1)/(gamma+1)) * (1-(1/MBaseL))* (1-(1/MBaseL)));

// Downstream slow acoustic wave amplification coefficient
var delta_pPrR_over_delta_pPrMiR = -(1- 2*MBaseR +1 / MBaseL/MBaseL) / (1+ 2*MBaseR +1 / MBaseL/ MBaseL);
(delta_pPrR_over_delta_pPrL,delta_pPrR_over_delta_pPrMiR)

Item1,2.154925878735875
Item2,-0.01484811194806477


compute the plot

In [47]:
var plot = new Plot2Ddata();
var EOCs= new List<double>();
public PlotFormat GetFormat(int count){
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Circle, PointTypes.OpenCircle, PointTypes.Diamond, PointTypes.OpenDiamond, PointTypes.UpperTriangle, PointTypes.OpenUpperTriangle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue,LineColors.Blue, LineColors.Black, LineColors.Black, LineColors.Red, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointType = PointTypes.OpenCircle;
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;
    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}
int count=0;
MultidimensionalArray errorsAndCells=MultidimensionalArray.Create(Cells.Length,degs.Length +1);
MultidimensionalArray errorsAndCellsIdeal=MultidimensionalArray.Create(Cells.Length,degs.Length+1);
for(int iSess =0;iSess < ses.Count();iSess++){
    var si = ses.Pick(iSess);
    //extract the relevant field and timestep Index
    var timesteps = si.Timesteps;
    var tEnd= timesteps.Last();
    var rho=tEnd.GetField("rho");
    int p= rho.Basis.Degree;
    if(p==0){
        continue;
    }
    int iCell= Cells.FirstIndexWhere(c => c==rho.Basis.GridDat.iGeomCells.Count);
    Console.Write($"P={p},cells={Cells[iCell]/3}");
    //extract AmpError
    if(si.Name.Contains("sP0.5")){
        double exact_amp_fac=delta_pPrR_over_delta_pPrMiR*1e-05; double tMin=15.8 +eps;double tMax=tMin+4.0-eps;
        if(tEnd.PhysicalTime>tMax){
            //List<double> ampErrors= ComputeAmpErrors(tMin,tMax, xMin, xMax, xRef,tRef,"p",si,false,exact_amp_fac,t0:9.95);
            double t=18.5;xMin=0.6;xMax=2.999;double t0=9.95; int xRef=800;
            (double[] xVals_dummy, double[] yVals, double tPhys) = GetPlotAlongTRay(xMin,xMax,t,xRef,"p",si,t0);
            var ex_amp=delta_pPrR_over_delta_pPrMiR*1e-5;
            errorsAndCells[iCell,p+1]=Math.Abs(yVals.Min()-ex_amp)/ex_amp;
            Console.WriteLine($" sp0.5, h: {Hs[iCell]}, error: {errorsAndCells[iCell,p+1]}, amp:{yVals.Min()} ");
        }
    }else{
        double exact_amp_fac=delta_pPrR_over_delta_pPrL*1e-05; double tMin=10.2 +eps;double tMax=tMin+1.0-eps;
        if(tEnd.PhysicalTime>tMax){
            double t=11.0;xMin=1.6;xMax=2.999;double t0=9.95; int xRef=800;
            (double[] xVals_dummy, double[] yVals, double tPhys) = GetPlotAlongTRay(xMin,xMax,t,xRef,"p",si,t0);
            var ex_amp=delta_pPrR_over_delta_pPrL*1e-5;
            errorsAndCells[iCell,p+1]=Math.Abs(yVals.Max()- ex_amp)/ ex_amp;
            Console.WriteLine($" sp1.5, h: {Hs[iCell]}, error: {errorsAndCells[iCell,p+1]}, amp:{yVals.Max()}");
        }
    }
}  


P=3,cells=121 sp1.5, h: 0.024793388429752067, error: 0.008435702090837617, amp:2.136747565995023E-05
P=3,cells=61 sp1.5, h: 0.04918032786885246, error: 0.01642093277528418, amp:2.1195399857454333E-05
P=3,cells=31 sp1.5, h: 0.0967741935483871, error: 0.029888334219064172, amp:2.090518733854907E-05
P=2,cells=121 sp1.5, h: 0.024793388429752067, error: 0.014977594168161112, amp:2.1226502734617014E-05
P=3,cells=16 sp1.5, h: 0.1875, error: 0.11132989488748639, amp:1.915018207165886E-05
P=2,cells=61 sp1.5, h: 0.04918032786885246, error: 0.0397271488306911, amp:2.0693168176322274E-05
P=2,cells=31 sp1.5, h: 0.0967741935483871, error: 0.057181665295065326, amp:2.031703628402326E-05
P=2,cells=16 sp1.5, h: 0.1875, error: 0.21439505255776334, amp:1.692920431706213E-05
P=1,cells=121 sp1.5, h: 0.024793388429752067, error: 0.03627579742141109, amp:2.0767542241006964E-05
P=1,cells=61 sp1.5, h: 0.04918032786885246, error: 0.07402738080698226, amp:1.9954023600998738E-05
P=1,cells=31 sp1.5, h: 0.096774193

In [49]:
var plot = new Plot2Ddata();
var EOCs= new List<double>();

MultidimensionalArray errorsAndCellsIdeal=MultidimensionalArray.Create(Cells.Length,degs.Length+1);
errorsAndCells.SetColumn(0,Hs.ToList());
errorsAndCellsIdeal.SetColumn(0,Hs.ToList());
int count=0;
foreach(int p in degs){
    errorsAndCellsIdeal.SetColumn(p+1,GetIdealConvPlot(errorsAndCells.GetColumn(p+1).ToList(),Hs.ToList(),p));
    EOCs.Add( ApproxSlope(errorsAndCells.GetColumn(p+1).ToList(),Hs.ToList()));
    plot.AddDataGroup($"P={p}",Hs,errorsAndCells.GetColumn(p+1), GetFormat(count));
    var fmt = GetFormat(count);
    fmt.PointSize=0.01;
    fmt.DashType=DashTypes.Dashed;
    plot.AddDataGroup($"ideal {p+1}",Hs,GetIdealConvPlot(errorsAndCells.GetColumn(p+1).ToList(),Hs.ToList(),p), fmt);
    count++;
}
errorsAndCells.SaveToTextFile("CNS_ConvergenceStudyErrors.txt");
errorsAndCellsIdeal.SaveToTextFile("CNS_ConvergenceStudyErrorsIdeal.txt");
plot.Xlabel="Hs";
plot.LogY = true;
plot.LogX = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Hs 
 
 
 
 
 P=0 
 
 
 P=0 
 
 
 
 
 
 
 ideal 1 
 
 
 ideal 1 
 
 
 
 
 
 
 P=1 
 
 
 P=1 
 
 
 
 
 
 
 
 
 
 
 ideal 2 
 
 
 ideal 2 
 
 
 
 
 
 
 
 
 
 
 P=2 
 
 
 P=2 
 
 
 
 
 
 
 
 
 
 
 ideal 3 
 
 
 ideal 3 
 
 
 
 
 
 
 
 
 
 
 P=3 
 
 
 P=3 
 
 
 
 
 
 
 
 
 
 
 ideal 4 
 
 
 ideal 4

In [50]:
EOCs

[ NaN, 1.3216942201044508, 1.3201503136622084, 1.2818542427667585 ]

In [ ]:
errorsAndCells.ToMsrMatrix().ToStringDense()

Assertion

In [ ]:
NUnit.Framework.Assert.IsTrue(EOCs[1]>1);

In [ ]:

NUnit.Framework.Assert.IsTrue(EOCs[2]>1);

In [ ]:

NUnit.Framework.Assert.IsTrue(EOCs[3]>1);